https://www.kaggle.com/tarobxl/magic-feature-v2-0-045-gain

https://www.kaggle.com/c/quora-question-pairs/discussion/33371

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict

In [2]:
train_orig =  pd.read_csv('data/train.csv', header=0)
test_orig =  pd.read_csv('data/test.csv', header=0)


In [3]:
df_id1 = train_orig[["qid1", "question1"]].drop_duplicates(keep="first").copy().reset_index(drop=True)
df_id2 = train_orig[["qid2", "question2"]].drop_duplicates(keep="first").copy().reset_index(drop=True)

df_id1.columns = ["qid", "question"]
df_id2.columns = ["qid", "question"]

print(df_id1.shape, df_id2.shape)

df_id = pd.concat([df_id1, df_id2]).drop_duplicates(keep="first").reset_index(drop=True)
print(df_id1.shape, df_id2.shape, df_id.shape)

((290654, 2), (299364, 2))
((290654, 2), (299364, 2), (537933, 2))


In [5]:
import csv
dict_questions = df_id.set_index('question').to_dict()
dict_questions = dict_questions["qid"]

new_id = 538000 # df_id["qid"].max() ==> 537933

def get_id(question):
    global dict_questions 
    global new_id 
    
    if question in dict_questions:
        return dict_questions[question]
    else:
        new_id += 1
        dict_questions[question] = new_id
        return new_id
    
rows = []
max_lines = 10
if True:
    with open('data/test.csv', 'r') as infile:
        reader = csv.reader(infile, delimiter=",")
        header = next(reader)
        header.append('qid1')
        header.append('qid2')
        
        if True:
            print(header)
            pos, max_lines = 0, 10*1000*1000
            for row in reader:
                # "test_id","question1","question2"
                question1 = row[1]
                question2 = row[2]

                qid1 = get_id(question1)
                qid2 = get_id(question2)
                row.append(qid1)
                row.append(qid2)

                pos += 1
                if pos >= max_lines:
                    break
                rows.append(row)

['test_id', 'question1', 'question2', 'qid1', 'qid2']


In [6]:
rows[0]

['0',
 'How does the Surface Pro himself 4 compare with iPad Pro?',
 'Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?',
 538001,
 538002]

In [7]:
with open('data/test_with_ids.csv', 'w') as infile:
    writer = csv.writer(infile, delimiter=",")
    writer.writerow(header)
    writer.writerows(rows)
    

In [6]:
DATA_DIR = "data/"

In [7]:
df_train = pd.read_csv(DATA_DIR + "train.csv", usecols=["qid1", "qid2"])

df_test = pd.read_csv(DATA_DIR + "test_with_ids.csv", usecols=["qid1", "qid2"])

df_all = pd.concat([df_train, df_test])

print("df_all.shape:", df_all.shape) # df_all.shape: (2750086, 2)

df = df_all

('df_all.shape:', (2750086, 2))


In [10]:
import networkx as nx

g = nx.Graph()

g.add_nodes_from(df.qid1)

edges = list(df[['qid1', 'qid2']].to_records(index=False))

g.add_edges_from(edges)

g.remove_edges_from(g.selfloop_edges())

print(len(set(df.qid1)), g.number_of_nodes()) # 4789604

print(len(df), g.number_of_edges()) # 2743365 (after self-edges)


(2454493, 4789604)
(2750086, 2743365)


In [11]:
df_output = pd.DataFrame(data=g.nodes(), columns=["qid"])

print("df_output.shape:", df_output.shape)

('df_output.shape:', (4789604, 1))


In [12]:
NB_CORES = 20

for k in range(2, NB_CORES + 1):

    fieldname = "kcore{}".format(k)

    print("fieldname = ", fieldname)

    ck = nx.k_core(g, k=k).nodes()

    print("len(ck) = ", len(ck))

    df_output[fieldname] = 0

    df_output.ix[df_output.qid.isin(ck), fieldname] = k

df_output.to_csv("data/cache/question_kcores.csv", index=None)

('fieldname = ', 'kcore2')
('len(ck) = ', 104340)
('fieldname = ', 'kcore3')
('len(ck) = ', 37019)
('fieldname = ', 'kcore4')
('len(ck) = ', 20580)
('fieldname = ', 'kcore5')
('len(ck) = ', 13552)
('fieldname = ', 'kcore6')
('len(ck) = ', 9949)
('fieldname = ', 'kcore7')
('len(ck) = ', 7674)
('fieldname = ', 'kcore8')
('len(ck) = ', 6156)
('fieldname = ', 'kcore9')
('len(ck) = ', 5124)
('fieldname = ', 'kcore10')
('len(ck) = ', 4190)
('fieldname = ', 'kcore11')
('len(ck) = ', 3606)
('fieldname = ', 'kcore12')
('len(ck) = ', 2979)
('fieldname = ', 'kcore13')
('len(ck) = ', 2529)
('fieldname = ', 'kcore14')
('len(ck) = ', 2167)
('fieldname = ', 'kcore15')
('len(ck) = ', 1807)
('fieldname = ', 'kcore16')
('len(ck) = ', 1633)
('fieldname = ', 'kcore17')
('len(ck) = ', 1523)
('fieldname = ', 'kcore18')
('len(ck) = ', 1270)
('fieldname = ', 'kcore19')
('len(ck) = ', 1125)
('fieldname = ', 'kcore20')
('len(ck) = ', 1024)


In [13]:
df_cores = pd.read_csv("data/cache/question_kcores.csv", index_col="qid")

df_cores.index.names = ["qid"]

df_cores['max_kcore'] = df_cores.apply(lambda row: max(row), axis=1)

df_cores[['max_kcore']].to_csv("data/cache/question_max_kcores.csv") # with index

In [4]:
cores_dict = pd.read_csv("data/cache/question_max_kcores.csv", index_col="qid").to_dict()["max_kcore"]


In [9]:
def gen_qid1_max_kcore(row):
    return cores_dict[row["qid1"]]
def gen_qid2_max_kcore(row):
    return cores_dict[row["qid2"]]

In [10]:
train_orig["qid1_max_kcore"] = df_train.apply(gen_qid1_max_kcore, axis=1)
test_orig["qid1_max_kcore"] = df_test.apply(gen_qid1_max_kcore, axis=1)


train_orig["qid2_max_kcore"] = df_train.apply(gen_qid2_max_kcore, axis=1)
test_orig["qid2_max_kcore"] = df_test.apply(gen_qid2_max_kcore, axis=1)


#df_train["max_kcore"] = df_train.apply(gen_max_kcore, axis=1)
#df_test["max_kcore"] = df_test.apply(gen_max_kcore, axis=1)

In [12]:
train_orig[["qid1_max_kcore","qid2_max_kcore"]].to_csv("data/max_kcore_train.csv",index=False)

In [13]:
test_orig[["qid1_max_kcore","qid2_max_kcore"]].to_csv("data/max_kcore_test.csv",index=False)

In [14]:
train_orig[["qid1_max_kcore","qid2_max_kcore"]].describe()

,qid1_max_kcore,qid2_max_kcore
count,404290.000000,404290.000000
mean,2.102231,2.168960
std,4.313840,4.372536
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,2.000000,2.000000
max,20.000000,20.000000
